# 1. Data Ingestion Module

In [4]:
import os
%pwd

'c:\\Projects\\011_NLP_HuggF_Summarizer\\TextSummarizer'

In [3]:
os.chdir("../")
%pwd


'c:\\Projects\\011_NLP_HuggF_Summarizer\\TextSummarizer'

## Basic Configuration

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path



In [7]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

## Configuration Updates

In [9]:
class ConfigurationManager:
    def __init__(self,
                 config_path = CONFIG_FILE_PATH,
                 params_path = PARAMS_FILE_PATH,
                 ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

## Components

In [10]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with the following info: {headers}")
        else:
            logger.info(f"File already exists")
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the destination directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Zip file extracted successfully")


In [ ]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)
data_ingestion.download_file()
data_ingestion.extract_zip_file()